In [9]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import InequalityToEquality, QuadraticProgramToQubo, LinearEqualityToPenalty
import numpy as np

n = 3 # personen
m = 4 # tasks

qubo = QuadraticProgram()

# define binary array of shape (n, m)
for k in range(n):
    for r in range(m):
        qubo.binary_var("x_"+str(k)+"_"+str(r))

# sample random costs per tasks
np.random.seed(seed=42)
c = np.random.rand(n, m)

### objective function
linear = {}

for k in range(n):
    for r in range(m):
        linear["x_"+str(k)+"_"+str(r)] = c[k, r]

qubo.minimize(linear=linear)

# constraint 1
for r in range(m):
    linear = {}
    for k in range(n):
        linear[f"x_{k}_{r}"] = 1
    qubo.linear_constraint(linear=linear, sense='EQ', rhs=1, name="constr_1_"+str(r))

# constraint 2
for k in range(n):
    linear = {}
    for r in range(m):
        linear[f"x_{k}_{r}"] = 1
    qubo.linear_constraint(linear=linear, sense='>=', rhs=1, name="constr_2_"+str(k))

print(qubo)

### convert inequality to equality constraints
ineq2eq = InequalityToEquality()
qubo_eq = ineq2eq.convert(qubo)

### add penalty term to constraints 
lineq2penalty = LinearEqualityToPenalty(penalty=50)
qubo_eq = lineq2penalty.convert(qubo_eq)

### convert integer variables to binary variables
qubo_eq = QuadraticProgramToQubo().convert(qubo_eq)
print(qubo_eq)

op, offset = qubo_eq.to_ising()
type(op)

print(f'Num of qubits {op.num_qubits}')


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.374540118847 x_0_0 + 0.950714306410 x_0_1 + 0.731993941811 x_0_2
      + 0.598658484197 x_0_3 + 0.156018640442 x_1_0 + 0.155994520336 x_1_1
      + 0.058083612168 x_1_2 + 0.866176145775 x_1_3 + 0.601115011743 x_2_0
      + 0.708072577796 x_2_1 + 0.020584494296 x_2_2 + 0.969909852162 x_2_3
Subject To
 constr_1_0: x_0_0 + x_1_0 + x_2_0 = 1
 constr_1_1: x_0_1 + x_1_1 + x_2_1 = 1
 constr_1_2: x_0_2 + x_1_2 + x_2_2 = 1
 constr_1_3: x_0_3 + x_1_3 + x_2_3 = 1
 constr_2_0: x_0_0 + x_0_1 + x_0_2 + x_0_3 >= 1
 constr_2_1: x_1_0 + x_1_1 + x_1_2 + x_1_3 >= 1
 constr_2_2: x_2_0 + x_2_1 + x_2_2 + x_2_3 >= 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_0_3 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_1_3 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
 0 <= x_2_3 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_0_3 x_1_0 x_1_1 x_1_2 x_1_3 x_2_0 x_2_1 x_2_2 x_2_3
E

### old qiskit aqua

In [ ]:
import time
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.operators import Z, I

from qiskit import Aer, BasicAer
from qiskit.aqua import QuantumInstance as AquaQuantumInstance
from qiskit.optimization.algorithms import MinimumEigenOptimizer

start_time = time.time()
backend = BasicAer.get_backend('qasm_simulator')
# backend = Aer.get_backend('statevector_simulator')

aqua_qinstance = AquaQuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

qaoa_mes = QAOA(quantum_instance=aqua_qinstance)

qaoa = MinimumEigenOptimizer(qaoa_mes)
# result = qaoa.compute_minimum_eigenvalue(op) # other method which is not working

qaoa_result = qaoa.solve(qubo_eq)
solution = qaoa_result.x

print(f'Time taken {time.time() - start_time}')
print()
print('Solution with slack variables')
solution

In [ ]:
total_num_vars = len(solution)
num_slack = total_num_vars - n * m

solution_wo_slack = solution[:-num_slack]
print('Solution without slack variables')
print(solution_wo_slack)
print('\nSolution feasbile?')
qubo.get_feasibility_info(solution_wo_slack)

In [ ]:
solution_reshaped = np.array(solution_wo_slack).reshape(n,m)

for person in range(n):
    for task in range(m):
        if solution_reshaped[person, task] == 1:
            print(f'Person {person} does task {task}')
        

### New Qiskit QAOA version

This does not work

In [ ]:
import time
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, SPSA
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit import  Aer

start_time = time.time()
backend = Aer.get_backend('qasm_simulator')

qinstance = QuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

qaoa_mes = QAOA(optimizer=COBYLA(), quantum_instance=qinstance)

qaoa = MinimumEigenOptimizer(qaoa_mes)
# result = qaoa.compute_minimum_eigenvalue(op) # other method which is not working

qaoa_result = qaoa.solve(qubo_eq)
solution = qaoa_result.x

print(f'Time taken {time.time() - start_time}')
print()
print('Solution with slack variables')
solution
